# Experiment on Vietnamese word segmentation

In [1]:
import gc
import os
import time


def collect(*, verbose=True):
    if verbose:
        print('garbage collector collected %d objects' % gc.collect())
    else:
        gc.collect()

collect()

garbage collector collected 0 objects


## Using Markdown programmatically

In [3]:
from IPython.display import Markdown, display


display(Markdown('*hello*, **world**!'))

*hello*, **world**!

## Vietnamese Word Segmentation

In [4]:
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: "%s"' % device)
if torch.cuda.is_available():
    print('Device name: "%s"' % torch.cuda.get_device_name(device))

Device: "cuda"
Device name: "NVIDIA GeForce RTX 3050 Laptop GPU"


In [5]:
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    pipeline
)


ws_tokenizer = AutoTokenizer.from_pretrained(
    'NlpHUST/vi-word-segmentation',
    truncate=True,
    model_max_length=512,
)
ws_model = AutoModelForTokenClassification.from_pretrained(
    'NlpHUST/vi-word-segmentation',
    torch_dtype=torch.float16,
    max_length=512,
)
ws = pipeline(
    'token-classification',
    model=ws_model,
    tokenizer=ws_tokenizer,
    device=device,
)

Device set to use cuda


In [6]:
def tokenize_vietnamese(
    texts: list[str],
    *,
    batch_size = 128,
) -> list[str]:
    if len(texts) == 0:
        return []
    global ws
    results = ws(texts, batch_size=batch_size)
    segments = []
    for doc in results:
        tokens = ''
        for e in doc:
            word = e['word']
            if '##' in word:
                tokens += word.replace('##', '')
            elif e['entity'] == 'I':
                tokens += '_' + word
            else:
                tokens += ' ' + word
        segments.append(tokens.lstrip())
    return segments

In [8]:
example = \
'''
Không ai có thể bị bắt làm nô lệ hoặc nô dịch; chế độ nô lệ và việc buôn nô lệ
bị cấm dưới mọi hình thức.
''' 
now = time.time()
tokenized_example = tokenize_vietnamese([example])[0]
delta = time.time() - now
print('delta (seconds): %.4f' % delta)

display(Markdown(
f'''
{example}

`{tokenized_example}`
'''
))

delta (seconds): 0.0360




Không ai có thể bị bắt làm nô lệ hoặc nô dịch; chế độ nô lệ và việc buôn nô lệ
bị cấm dưới mọi hình thức.


`Không ai có_thể bị bắt làm nô_lệ hoặc nô_dịch ; chế_độ nô_lệ và việc buôn nô_lệ bị cấm dưới mọi hình_thức .`
